In [1]:
import json
import ctypes
from dask.distributed import Client
import dask.dataframe as dd

def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)


In [2]:
client = Client()
# Helps fix any memory leaks.
client.run(trim_memory)
client = client.restart()


2024-01-24 00:53:27,433 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-o8f_k029', purging
2024-01-24 00:53:28,022 - distributed.nanny - WARNING - Restarting worker
2024-01-24 00:53:28,041 - distributed.nanny - WARNING - Restarting worker
2024-01-24 00:53:28,044 - distributed.nanny - WARNING - Restarting worker
2024-01-24 00:53:28,052 - distributed.nanny - WARNING - Restarting worker


In [3]:
user_reviews_ddf = dd.read_csv('user_reviews.csv')
user_reviews_ddf

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
npartitions=445,,,,,,,,,
,object,object,object,object,object,float64,object,float64,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [4]:
user_reviews_ddf['number_products_rated'] = user_reviews_ddf['asin']

In [5]:
user_reviews_ddf['avg_ratings'] = user_reviews_ddf['overall']


In [6]:
# Extract the year
user_reviews_ddf['reviewing_since'] = user_reviews_ddf['reviewTime'].str.slice(-4).astype("int")



In [7]:

# Assuming user_reviews_ddf is your Dask DataFrame
# First, extract the number of helpful votes from the 'helpful' column
user_reviews_ddf['helpful_votes'] = user_reviews_ddf['helpful'].apply(lambda x: eval(x)[0], meta=('x', 'int'))



In [8]:


# Assuming user_reviews_ddf is your Dask DataFrame
# First, extract the number of helpful votes from the 'helpful' column
user_reviews_ddf['total_votes'] = user_reviews_ddf['helpful'].apply(lambda x: eval(x)[1], meta=('x', 'int'))



In [ ]:
result_df = user_reviews_ddf.groupby("reviewerID", sort=False).agg({"number_products_rated":"count", "avg_ratings":"mean", "reviewing_since":"min", "helpful_votes":"sum", "total_votes":"sum"}, split_out = 8)


submit = result_df.describe().compute().round(2)    
with open('results_PA0.json', 'w') as outfile: 
    json.dump(json.loads(submit.to_json()), outfile)